# History of the Relations Between Supervisors and Students

In [1]:
import pandas as pd

In [2]:
supervision_df = pd.read_csv('relations.csv')
supervision_df.drop_duplicates(inplace=True)
supervision_df.reset_index(drop=True, inplace=True)
supervision_df.head(10)

,supervisor,student
0,Michel Mayor,Didier Queloz
1,Gérard Mourou,Donna Strickland
2,Rainer Weiss,Nergis Mavalvala
3,Rainer Weiss,Bruce Allen (physicist)
4,Rainer Weiss,Sarah Veatch
5,Jerrold R. Zacharias,Rainer Weiss
6,A. Carl Helmholz,Barry C. Barish
7,Kip S. Thorne,William L. Burke
8,Kip S. Thorne,Carlton M. Caves
9,Kip S. Thorne,Lee Samuel Finn


Number of unique people in the dataset

In [10]:
len(set(supervision_df.student.tolist() + supervision_df.supervisor.tolist()))

7552

Number of unique students in the dataset

In [13]:
len(supervision_df.student.unique())

6938

Number of unique supervisors in the dataset

In [14]:
len(supervision_df.supervisor.unique())

3256